# Layers and Blocks

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2953, -0.0544, -0.0325, -0.1741, -0.2114,  0.0400, -0.0546,  0.2308,
         -0.0946,  0.0859],
        [ 0.3654, -0.0045, -0.1742, -0.2506, -0.0385,  0.1261, -0.0493,  0.3270,
         -0.0750,  0.2002]], grad_fn=<AddmmBackward>)

## A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self, X):
        x = self.hidden(X)
        x = F.relu(x)
        out = self.out(x)
        return out

In [3]:
net = MLP()
net(X)

tensor([[-0.0468, -0.0199, -0.0095,  0.0778, -0.2067, -0.0272, -0.1152,  0.0278,
         -0.3658,  0.0542],
        [ 0.0241, -0.0958,  0.1100,  0.0662, -0.1539, -0.0830, -0.0024, -0.0107,
         -0.2227,  0.0725]], grad_fn=<AddmmBackward>)

## Sequential Block

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, arg in enumerate(args):
            self._modules[str(idx)] = arg
        
    def forward(self, X):
        for k,v in self._modules.items():
            X = v(X)
        return X
    

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0513,  0.0978,  0.1923,  0.1167,  0.2786, -0.0286,  0.0760,  0.0567,
          0.1060, -0.1365],
        [ 0.1034,  0.0460,  0.2194,  0.0364,  0.0870,  0.0970,  0.0172,  0.0426,
          0.0149, -0.2083]], grad_fn=<AddmmBackward>)

## Executing Code in the Forward Propagation Function

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)

        while X.abs().sum() > 1:
            X /=2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2528, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(), nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self, X):
        return self.linear(self.net(X))
        
net = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP())
net(X)

tensor(-0.0679, grad_fn=<SumBackward0>)

## Exercise

In [9]:
class ConcatMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net1 = nn.Sequential(nn.Linear(20, 20), nn.ReLU())
        self.net2 = nn.Sequential(nn.Linear(20, 40), nn.ReLU(), nn.Linear(40,20), nn.ReLU())
        
    def forward(self, X):
        return torch.cat((self.net1(X), self.net2(X)),1)

net = ConcatMLP()
net(X).shape

torch.Size([2, 40])

In [11]:
class NConcatMLP(nn.Module):
    def __init__(self, n=1):
        super().__init__()
        for i in range(n):
            self._modules[str(i)] = nn.Sequential( nn.Linear(20,20), nn.ReLU(), nn.Linear(20,10),nn.ReLU())

    def forward(self, X):
        out = None
        for module in self._modules.values():
            if out is None:
                out = module(X)
            else:
                out = torch.cat((out, module(X)),1)
        
        return out

net = NConcatMLP(n=3)
net(X).shape

torch.Size([2, 30])

In [12]:
print(net)

NConcatMLP(
  (0): Sequential(
    (0): Linear(in_features=20, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): ReLU()
  )
  (1): Sequential(
    (0): Linear(in_features=20, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): ReLU()
  )
  (2): Sequential(
    (0): Linear(in_features=20, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): ReLU()
  )
)
